In [1]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    get_max_part,
    get_bigquery_client,   
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [ ]:
from google.cloud.bigquery.job import QueryJobConfig

In [2]:
from datetime import datetime, timedelta

In [92]:
execution_dt = datetime.strptime(current_dt, '%Y-%m-%d')
execution_dt_next = (execution_dt + timedelta(days=1))
current_dt_next = execution_dt_next.strftime('%Y-%m-%d')
ttl = int(ttl)
print(f'execution_dt: {current_dt}')
print(f"state: {state}")
print(f"ttl: {ttl}")

execution_dt: 2024-07-08
state: stg
ttl: 30


In [106]:
""" no partition data in comm (snapshot) """
db_name  = "comm"
tmbr_meta_tbl= "mp_taxonomies_brand" 
tmbr_meta_map_tbl = "mp_taxonomies_brandMapp"

In [126]:
temp_db = 'temp_1d'

In [107]:
save_db_name = 'adot_reco_dev'
save_meta_table = 'tmbr_meta_mapping_tbl'

In [108]:
bq_clinet = get_bigquery_client()

### UPDATED META 임시테이블에 저장


In [ ]:
#recent_meta_dt = get_max_part(f"{PROJECT_ID}.comm.{tmbr_meta_tbl}")

In [159]:
## Meta table 전처리 로직: 중복 product 처리 (제거 x -> 메타 합산)
query = f"""

WITH META_TABLE AS (
    SELECT  brand_id,
            count(distinct brand_name) over(partition by COALESCE(TRIM(LOWER(brand_name)), '')) as brand_cnt,
            COALESCE(TRIM(LOWER(brand_name)), '') as brand_name,
            del_yn,
            display_name
            prm_brand_name,
            categories,
            COALESCE(TRIM(LOWER(category_large_name)), '') as category_large_name,
            COALESCE(TRIM(LOWER(category_medium_name)), '') as category_medium_name,
            COALESCE(TRIM(LOWER(category_small_name)), '') as category_small_name,
            COALESCE(TRIM(LOWER(description)), '') as description

    FROM    {db_name}.{tmbr_meta_tbl}
    WHERE   brand_name is not null and brand_name !='' and LOWER(brand_name) NOT LIKE '%test%'
),

DUPLICATED_BRAND_TABLE AS(
    SELECT  brand_name, 
            ARRAY_AGG(category_large_name) as category_large_names,
            ARRAY_AGG(category_medium_name) as category_medium_names,
            ARRAY_AGG(category_small_name) as category_small_names,
            ARRAY_AGG(description) as descriptions

    FROM (
        SELECT  brand_name,
                category_large_name,
                category_medium_name,
                category_small_name, 
                description

        FROM META_TABLE
        WHERE brand_name !='' AND brand_cnt > 1 
    )
    GROUP BY brand_name
),

NON_DUPLICATED_BRAND_TABLE AS(
    SELECT  brand_name,
            category_large_name,
            category_medium_name,
            category_small_name, 
            description
            

    FROM META_TABLE
    WHERE brand_name !='' AND brand_cnt = 1
),

MERGED_DUPLICATE_BRAND_TABLE AS(
    SELECT  brand_name,
            ARRAY_TO_STRING(category_large_names, ',') as category_large_name,
            ARRAY_TO_STRING(category_medium_names, ',') as category_medium_name,
            ARRAY_TO_STRING(category_small_names, ',') as category_small_name,
            ARRAY_TO_STRING(descriptions, ',') as description,
            
    FROM(
        SELECT  brand_name,
                ARRAY(
                    SELECT category_large_name
                    FROM UNNEST(category_large_names) AS category_large_name 
                    WHERE category_large_name != ''
                  ) AS category_large_names,
                ARRAY(
                    SELECT category_medium_name 
                    FROM UNNEST(category_medium_names) AS category_medium_name 
                    WHERE category_medium_name != ''
                  ) AS category_medium_names,

                ARRAY(
                    SELECT category_small_name
                    FROM UNNEST(category_small_names) AS category_small_name
                    WHERE category_small_name != ''
                  ) AS category_small_names,
                ARRAY(
                    SELECT description
                    FROM UNNEST(descriptions) AS description
                    WHERE description != ''
                  ) AS descriptions     

        FROM DUPLICATED_BRAND_TABLE
    )
)

SELECT  *
        
FROM MERGED_DUPLICATE_BRAND_TABLE

UNION ALL

SELECT  *

FROM NON_DUPLICATED_BRAND_TABLE
"""


In [161]:
job_config = QueryJobConfig()
temp_no_duplicate_meta_table = f'{PROJECT_ID}.{temp_db}.tmbr_temp_table'
job_config.destination = temp_no_duplicate_meta_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [162]:
# 임시 테이블 생성
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
WITH META_TABLE AS (
    SELECT  *
    FROM {temp_no_duplicate_meta_table}
),

SUB_WORD_TABLE AS (

    SELECT  brand_word,
            brand_name,
            category_large_name,
            category_medium_name,
            category_small_name,
            description,
            count(brand_word) over(partition by brand_word) as word_cnt
            
    FROM (
        SELECT  category_large_name,
                category_medium_name,
                category_small_name,
                description,
                brand_name,
                brand_word
                
        FROM META_TABLE,
        UNNEST(SPLIT(brand_name, ' ')) AS brand_word
    )
),

tmbr_etymology_table AS (
    SELECT  A.brand_name,
            A.category_large_name,
            A.category_medium_name,
            A.category_small_name,
            A.description
    FROM (

        SELECT distinct brand_name,
               category_large_name,
               category_medium_name,
               category_small_name,
               description

               
        FROM SUB_WORD_TABLE
        WHERE brand_name = brand_word
    )AS A
    LEFT JOIN  (
        SELECT  distinct COALESCE(TRIM(LOWER(brand_name)), '') as brand_name

        FROM  {db_name}.{tmbr_meta_tbl}
        WHERE brand_name is not null and brand_name !=''
    ) AS B
    ON A.brand_name = B.brand_name
),

derivative_table AS(
    SELECT  distinct    brand_word as brand_name,
                        brand_name as derivative_brand
    FROM SUB_WORD_TABLE
    WHERE brand_word in (SELECT distinct brand_name FROM tmbr_etymology_table) 
    AND brand_name != brand_word AND brand_word !=''
),

final_tmbr_etymology_table AS (

    SELECT distinct COALESCE(brand_name, '') as brand_name,
                    COALESCE(description, '') as description,
                    COALESCE(derivative_brand, '') as derivative_brand,
                    PARSE_DATE('%Y-%m-%d', '{current_dt}') as dt 
    FROM (
        SELECT  A.brand_name,
                A.category_large_name,
                A.category_medium_name,
                A.category_small_name,
                A.description,
                B.derivative_brand

        FROM tmbr_etymology_table AS A

        LEFT JOIN (
            SELECT  brand_name,
                    derivative_brand
            FROM derivative_table
        ) AS B
        ON A.brand_name = B.brand_name
    )
)
SELECT * 
FROM final_tmbr_etymology_table A
"""

In [ ]:
try:
    etymology_max_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.tmbr_etymology_table")
    query = query + f"""WHERE NOT EXISTS (
                SELECT 1
                FROM {PROJECT_ID}.{save_db_name}.tmbr_etymology_table AS B
                WHERE A.brand_name = B.brand_name AND A.description = B.description
                      AND A.derivative_brand = B.derivative_brand 
                AND dt <= '{etymology_max_dt}'
            )"""
except ValueError as e:
    print('first time')
    pass

In [ ]:
## 증분 관리 테이블
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_etymology_table', partition='dt')

## Category mapping

In [ ]:
query = f"""
WITH tmbr_etymology_table AS (
    SELECT distinct *
    FROM {temp_no_duplicate_meta_table}
)

SELECT *
FROM (
    SELECT 
        brand_name,
        category_large_name,
        category_medium_name,
        category_small_name,
        description,
        PARSE_DATE('%Y-%m-%d', '{current_dt}') as dt 

    FROM  tmbr_etymology_table
    WHERE category_small_name !='' OR category_medium_name!=''
) A
"""

In [ ]:
try:
    category_mapping_basic_table_max_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.tmbr_basic_category_mapping_table")
    query = query + f"""WHERE NOT EXISTS (
                SELECT 1
                FROM {PROJECT_ID}.{save_db_name}.tmbr_basic_category_mapping_table AS B
                WHERE A.brand_name = B.brand_name AND A.category_medium_name = B.category_medium_name AND A.category_small_name = B.category_small_name
                AND dt <= '{category_mapping_basic_table_max_dt}'
            )"""
except ValueError as e:
    print('first time')
    pass

In [ ]:
## 증분 관리 테이블
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_basic_category_mapping_table', partition='dt')

In [179]:
query = f"""
WITH tmbr_category_mapping_table AS (
    SELECT distinct *
    FROM {PROJECT_ID}.{save_db_name}.tmbr_basic_category_mapping_table AS A
    WHERE dt <= '{current_dt}'
)

SELECT  brand_name,
        category_large_name,
        category_medium_name,
        category_small_name,
        description
    
FROM  tmbr_category_mapping_table
WHERE category_medium_name !='' OR category_small_name !=''
"""

In [180]:
categories_mapping_df = bq_to_pandas(query)

unsupported operand type(s) for /: 'NoneType' and 'int'
Downloading: 100%|██████████|


In [181]:
categories_large_mapping = categories_mapping_df.set_index('brand_name').to_dict()['category_large_name']
categories_mediuam_mapping = categories_mapping_df.set_index('brand_name').to_dict()['category_medium_name']
categories_small_mapping = categories_mapping_df.set_index('brand_name').to_dict()['category_small_name']

In [182]:
import json
categories_large_mapping_json = json.dumps(categories_large_mapping, ensure_ascii=False)
categories_medium_mapping_json = json.dumps(categories_mediuam_mapping, ensure_ascii=False)
categories_small_mapping_json = json.dumps(categories_small_mapping, ensure_ascii=False)

In [199]:
query = f"""
CREATE TEMP FUNCTION get_meiduam_caegory (brand_name STRING)
RETURNS STRING
LANGUAGE js AS '''
  var medium_category_mapping_json = {categories_medium_mapping_json};

  return (brand_name in medium_category_mapping_json) ? medium_category_mapping_json[brand_name] : '';
''';

CREATE TEMP FUNCTION get_small_caegory (brand_name STRING)
RETURNS STRING
LANGUAGE js AS '''
  var small_category_mapping_json = {categories_small_mapping_json};

  return (brand_name in small_category_mapping_json) ? small_category_mapping_json[brand_name] : '';
''';

WITH META_TABLE AS (
    SELECT *
    FROM {PROJECT_ID}.{save_db_name}.tmbr_basic_category_mapping_table AS A
    WHERE dt <= '{current_dt}'
),

tmbr_etymology_category_table AS (
    SELECT distinct *
    FROM (
        SELECT  brand_name,
                derivative_brand,
                CASE 
                    WHEN medium_category_brand = '' AND medium_category_derivative_brand = '' THEN ''
                    WHEN medium_category_brand = '' AND medium_category_derivative_brand != '' THEN medium_category_derivative_brand
                    ELSE medium_category_brand
                END AS meidum_category,
                CASE 
                    WHEN small_category_brand = '' AND small_category_derivative_brand = '' THEN ''
                    WHEN small_category_brand = '' AND small_category_derivative_brand != '' THEN small_category_derivative_brand
                    ELSE small_category_brand
                END AS small_category,
                description
        FROM (
            SELECT  brand_name,
                    derivative_brand,
                    get_meiduam_caegory(brand_name) as medium_category_brand,
                    get_meiduam_caegory(derivative_brand) as medium_category_derivative_brand,
                    get_small_caegory(brand_name) as small_category_brand,
                    get_small_caegory(derivative_brand) as small_category_derivative_brand,
                    description

            FROM {PROJECT_ID}.{save_db_name}.tmbr_etymology_table
            WHERE dt <= '{current_dt}'
        )
    )
    WHERE (small_category!='') OR (meidum_category!='')
)
-- 여기 자업
SELECT *
FROM (
    SELECT  brand_name,
            category_medium_name as meidum_category,
            category_small_name as small_category,
            description
    FROM  META_TABLE
    WHERE brand_name not in (SELECT distinct brand_name FROM tmbr_etymology_category_table)
)
WHERE (meidum_category!='') OR (small_category!='')

UNION ALL
(
    SELECT  brand_name,
            meidum_category,
            small_category,
            description
                    
    FROM tmbr_etymology_category_table

    UNION ALL

    SELECT  derivative_brand as brand_name,
            meidum_category,
            small_category,
            description 

    FROM tmbr_etymology_category_table
)
"""

In [200]:
job_config = QueryJobConfig()
temp_meta_preprocessed_table = f'{PROJECT_ID}.{temp_db}.tmbr_meta_preprocessed_table'
job_config.destination = temp_meta_preprocessed_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [201]:
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

# AGGREGATE

In [225]:
query = f"""
WITH  categories_count_table AS (
  SELECT 
    brand_name,
    meidum_category,
    small_category,
    COUNT(*) as category_count
  FROM {PROJECT_ID}.{temp_db}.tmbr_meta_preprocessed_table
  GROUP BY brand_name, meidum_category, small_category
),

ranked_items AS (
  SELECT 
    brand_name,
    meidum_category,
    small_category,
    category_count,
    ROW_NUMBER() OVER (PARTITION BY brand_name ORDER BY category_count DESC) as rank
  FROM categories_count_table
)
SELECT *
FROM ranked_items
"""

In [ ]:
job_config = QueryJobConfig()
temp_tmbr_rank_item = f'{PROJECT_ID}.{temp_db}.tmbr_temp_rank_table'
job_config.destination = temp_tmbr_rank_item
job_config.write_disposition = 'WRITE_TRUNCATE'

In [ ]:
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
WITH ranked_items AS (
  SELECT *
  FROM {PROJECT_ID}.{temp_db}.tmbr_temp_rank_table
)
SELECT  distinct  brand_name,
                  meidum_categories,
                  small_categories,
                  PARSE_DATE('%Y-%m-%d', '{current_dt}') as dt 
FROM (
    SELECT brand_name,
           ARRAY_TO_STRING(ARRAY_AGG(DISTINCT meidum_category), ',') AS meidum_categories,
           ARRAY_TO_STRING(ARRAY_AGG(DISTINCT small_category), ',') AS small_categories
    FROM ranked_items
    WHERE rank = 1 or category_count > 1
    GROUP BY brand_name 
) AS A
"""

In [ ]:
try:
    category_mapping_table_max_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.tmbr_final_category_mapping_table")
    query = query + f"""WHERE NOT EXISTS (
                SELECT 1
                FROM {PROJECT_ID}.{save_db_name}.tmbr_final_category_mapping_table AS B
                WHERE A.brand_name = B.brand_name AND A.meidum_categories = B.meidum_categories AND A.small_categories = B.small_categories
                AND dt <= '{category_mapping_table_max_dt}'
            )"""
except ValueError as e:
    print('first time')
    pass

In [ ]:
## 증분 관리 테이블
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_final_category_mapping_table', partition='dt')

## ACTIVE + DEL_YN 적용한 현재 사용 카테고리

In [ ]:
query = f"""
SELECT  B.brand_id,
        A.brand_name,
        A.meidum_categories,
        A.small_categories,
        REPLACE(TRIM(LOWER(A.brand_name)), ' ', '') as brand_name_model,
        B.description,
        B.updated_at,
        B.created_at,
        B.del_yn,
        B.brand_status_type,
        PARSE_DATE('%Y-%m-%d', '{current_dt_next}') as dt 
FROM (
    SELECT brand_name,
           ARRAY_TO_STRING(ARRAY_AGG(DISTINCT meidum_category), ',') AS meidum_categories,
           ARRAY_TO_STRING(ARRAY_AGG(DISTINCT small_category), ',') AS small_categories
    FROM  {PROJECT_ID}.{temp_db}.tmbr_temp_rank_table
    WHERE rank = 1 or category_count > 1
    GROUP BY brand_name 
) AS A

LEFT JOIN (
    SELECT *
    FROM  {db_name}.{tmbr_meta_tbl}
    WHERE brand_status_type = 'ACTIVE'
    AND   del_yn = 'N'
) AS B
ON A.brand_name = TRIM(LOWER(B.brand_name))
WHERE B.brand_id is not null
"""

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_meta_active_tbl', partition='dt')

In [ ]:
ttl_query = f"""
ALTER TABLE
  {PROJECT_ID}.{save_db_name}.tmbr_meta_active_tbl
SET
  OPTIONS(partition_expiration_days={ttl})
"""
bq_clinet.query(ttl_query).result()

# drop temp table code

In [ ]:
for temp_table_name in ["tmbr_etymology_table", "tmbr_meta_preprocessed_table", "tmbr_temp_rank_table", "tmbr_temp_table"]:
    drop_query = f"""
    DROP TABLE IF EXISTS {PROJECT_ID}.{temp_db}.{temp_table_name}
    """
    bq_clinet.query(drop_query).result()